In [33]:
import json
import os
import pandas as pd
import pickle
import pyarrow.parquet as pq
import re


# Lectura archivo fausto

In [34]:
ruta_absoluta6 = 'archivos_parquet_florida/2_ETL_FLORIDA_FILTRADO_x_RESTO_x_MAPA_x_TEXTO.parquet'


# Obtener la ruta absoluta al archivo Parquet
ruta_absoluta6 = os.path.abspath(ruta_absoluta6)

# Verificar si el archivo existe
if os.path.exists(ruta_absoluta6):
    # Cargar el archivo Parquet en un DataFrame
    df6 = pd.read_parquet(ruta_absoluta6)

    # Ahora puedes trabajar con el DataFrame df
    print(df6)
else:
    print(f"El archivo {ruta_absoluta6} no existe.")

                         business_name  \
1                   Honduras Nica Cafe   
2                   Honduras Nica Cafe   
3                   Honduras Nica Cafe   
4                   Honduras Nica Cafe   
5                   Honduras Nica Cafe   
...                                ...   
1005932  Top Taste Jamaican Restaurant   
1005933  Top Taste Jamaican Restaurant   
1005934  Top Taste Jamaican Restaurant   
1005935  Top Taste Jamaican Restaurant   
1005936  Top Taste Jamaican Restaurant   

                                                                     address  \
1                        Honduras Nica Cafe, 280 NW 79th St, Miami, FL 33150   
2                        Honduras Nica Cafe, 280 NW 79th St, Miami, FL 33150   
3                        Honduras Nica Cafe, 280 NW 79th St, Miami, FL 33150   
4                        Honduras Nica Cafe, 280 NW 79th St, Miami, FL 33150   
5                        Honduras Nica Cafe, 280 NW 79th St, Miami, FL 33150   
...            

In [35]:
df6.shape

(616166, 16)

In [36]:
df6.columns

Index(['business_name', 'address', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC',
       'state', 'user_name', 'time', 'rating', 'text'],
      dtype='object')

In [37]:
df6.state.unique()

array(['Closes soon ⋅ 9PM ⋅ Opens 7AM Fri', 'Open ⋅ Closes 1AM',
       'Open ⋅ Closes 10PM', 'Closes soon ⋅ 8PM ⋅ Opens 11AM Fri',
       'Open ⋅ Closes 10:30PM', 'Permanently closed', 'Open ⋅ Closes 9PM',
       'Open ⋅ Closes 8PM', 'Open ⋅ Closes 7:30PM', 'Temporarily closed',
       'Closed ⋅ Opens 12PM', 'Closed ⋅ Opens 11AM',
       'Closed ⋅ Opens 6:30PM', 'Closed ⋅ Opens 4PM',
       'Closed ⋅ Opens 10PM', None, 'Open 24 hours',
       'Closed ⋅ Opens 10AM', 'Closed ⋅ Opens 11AM Thu',
       'Closed ⋅ Opens 10AM Thu', 'Closed ⋅ Opens 7AM Thu',
       'Closed ⋅ Opens 8:30AM Thu',
       'Closes soon ⋅ 9PM ⋅ Opens 10:30AM Thu', 'Open now',
       'Open ⋅ Closes 9:30PM', 'Open ⋅ Closes 7PM', 'Open ⋅ Closes 12AM',
       'Closed ⋅ Opens 12AM Thu', 'Open ⋅ Closes 4PM',
       'Closed ⋅ Opens 11:30AM', 'Open ⋅ Closes 6PM',
       'Closed ⋅ Opens 12:30PM', 'Opens soon ⋅ 8AM',
       'Closed ⋅ Opens 11AM Wed', 'Open ⋅ Closes 11PM',
       'Closed ⋅ Opens 10AM Wed', 'Closed ⋅ Opens 7PM 

In [38]:
# Saco los registros de restaurantes ya cerrados

df6 = df6[df6["state"]!= "Permanently closed"]

In [39]:
df6.shape

(580926, 16)

Estos estan cerrados, si bien puede ser que se re ubicaron o cerraron completamente, las reviews crreo que no habria que contemplarlas, ya que la experiencia en ese lugar y con esa atencion fue recibida en esos establecimientos. Por ejemplo Sarku sigue vigente pero en otro lugares, casa lario cerro, etc

In [40]:
restaurants = df6.business_name.value_counts()
restaurants

McDonald's                          36700
Wendy's                             11079
Chick-fil-A                          9248
Cracker Barrel Old Country Store     8412
IHOP                                 8107
                                    ...  
Coral Gate Café                         3
Broadway Cafe                           3
La Zeta Mexicana                        3
Antojos Boricuas II                     3
La Lunita Coffee Shop                   2
Name: business_name, Length: 2894, dtype: int64

In [41]:
restaurants[restaurants>10]

McDonald's                            36700
Wendy's                               11079
Chick-fil-A                            9248
Cracker Barrel Old Country Store       8412
IHOP                                   8107
                                      ...  
Bodega Olé                               11
The Taco Joint                           11
The Bang Shack                           11
Odaly's Delight Café                     11
La Nueva Banana Restaurant & grill       11
Name: business_name, Length: 2347, dtype: int64

Hay algunos restaurantes que son cadenas como podemos ver arriba

In [42]:
df6.category

1          ['Honduran restaurant']
2          ['Honduran restaurant']
3          ['Honduran restaurant']
4          ['Honduran restaurant']
5          ['Honduran restaurant']
                    ...           
1005897     ['Italian restaurant']
1005898     ['Italian restaurant']
1005899     ['Italian restaurant']
1005900     ['Italian restaurant']
1005901     ['Italian restaurant']
Name: category, Length: 580926, dtype: object

In [43]:
# Explode the column into separate rows
df_exploded = df6.explode('category')

# Perform value counts to see all the categories
value_counts = df_exploded['category'].value_counts().reset_index()
value_counts.columns = ['Category', 'Count']

# Display the result
print(value_counts)

                                                                                                                     Category  \
0         ['Fast food restaurant' 'Breakfast restaurant' 'Coffee shop'\n 'Hamburger restaurant' 'Restaurant' 'Sandwich shop']   
1                                                                                                              ['Restaurant']   
2                                                                ['Fast food restaurant' 'Hamburger restaurant' 'Restaurant']   
3                                                                                                      ['Seafood restaurant']   
4     ['Fast food restaurant' 'Breakfast restaurant' 'Caterer'\n 'Chicken restaurant' 'Delivery service' 'Family restaurant']   
...                                                                                                                       ...   
1390                                                                                ['Cuban resta

In [44]:
df_exploded.category.unique()

array(["['Honduran restaurant']", "['Restaurant']",
       "['Jamaican restaurant']", ..., "['Restaurant' 'Bar' 'Cafe']",
       "['Seafood restaurant' 'Cajun restaurant']",
       "['Ice cream shop' 'Churreria' 'Coffee shop' 'Hot dog restaurant'\n 'Sandwich shop' 'Soft drinks shop']"],
      dtype=object)

Van estas categorias??

- Caterer
- 'Fruit and vegetable store'
-  'Juice shop'
- 'Event planner'
- 'Party equipment rental service'
- 'Gas station'
- 'Alternative fuel station'
- 'ATM'
- 'Convenience store'
- 'Electric vehicle charging station'
- 'Catering'
- 'Fire protection service'
- HVAC contractor
- 'Restaurant supply store'
- Appliance parts supplier
- 'Appliance store'
- 'Commercial refrigerator supplier'
- 'Industrial equipment supplier'
- 'Kitchen supply store'
- 'Small appliance repair service'
- 'Used appliance store'
- Wellness center
- 'Acupuncture clinic'}
- 'Alternative medicine practitioner'
- 'Fitness center'
- 'Hair salon'
- 'Holistic medicine practitioner
- 'Meditation center
- 'Pilates studio'
- 'Yoga studio'
- 'Health spa'
- 'Spa and health club'
- 'Grocery store'
- 'Book store'
- 'Vitamin & supplements store'
- 'Car wash'
- 'Banner store'
- 'Digital printing service'
- 'Print shop'
-  'Market
- 'Dollar store'
- 'Discount store'
- 'General store'
- 'Home goods store'
- 'Office supply store'
- 'Party store'
- 'School supply store'
- 'Variety store'
- 'Software company'
- 'Accounting software company'
- 'Business to business service'
- 'Computer networking center'
- 'Financial institution'
-  'Website designer'
- 'Performing arts group'
- 'Service establishment'
- 'Mobile caterer'
- 'Box lunch supplier'
- 'Indoor playground'
-  "Children's party service"
- 'Park'
- 'Playground'
- 'Video arcade'
- 'Manufacturer'
- 'Horseback riding service'
- 'Pony ride service'
- 'Fresh food market'
- 'Tourist attraction'
- 'Weight loss service'
-  'Eating disorder treatment center'
- 'Wi-Fi spot'
- 'Butcher shop'
- 'Hotel'
-  'Resort hotel'
- 'Dance club'
- 'Airline ticket agency'
-  'Courier service'
- 'Gift shop'
- 'Money transfer service'
-   'Shipping service'
- 'Gym'
- 'Gymnastics center'
- 'Parkour spot'
- 'Rock climbing gym'
- 'Market operator'
- 'Grocery delivery service'
-   'Appliance repair service'
- 'Store'
- 'Condiments supplier'
- 'Recreation center'
- 'Stadium'
- 'Plaza'
- "Children's amusement center"
- 'Art gallery'
- 'Musical club'
- 'Rock music club'
-   'Wellness program'
- 'Liquor store'
-   'Candy store'
- 'Gas company'
- 'Wedding venue'
- 'Hookah store'
- 'Party planner'
- 'Produce wholesaler'
- 'Wholesale grocer'
-  'Drive-in movie theater'
- 'Coaching center'
- 'Chocolate shop'
- 'Corporate office'
-  'Flower delivery'
- 'Milk delivery service'
- 'Adult entertainment club'
- 'Boutique'
- 'Clothing store'
-   'Furniture store'
- 'Vintage clothing store'
- 'Shopping mall'
-   'Movie theater'
- 'Wholesale market'
- 'Dry ice supplier'
-   'Chemical wholesaler'
- 'Industrial gas supplier'
- 'Stage'
-   'Amusement center'
- 'Wedding buffet'
- Food producer
     

In [45]:
categories_not_to_include = ["Caterer", 'Fruit and vegetable store', 'Juice shop', 'Event planner', 'Party equipment rental service', 'Gas station', 'Alternative fuel station', 'ATM', 'Convenience store', 'Electric vehicle charging station', 'Catering', 'Fire protection service', 'HVAC contractor', 'Restaurant supply store',
'Appliance parts supplier', 'Appliance store', 'Commercial refrigerator supplier', 'Industrial equipment supplier', 'Kitchen supply store', 'Small appliance repair service', 'Used appliance store', 'Wellness center', 'Acupuncture clinic', 'Alternative medicine practitioner', 'Fitness center',
'Hair salon', 'Holistic medicine practitioner', 'Meditation center', 'Pilates studio', 'Yoga studio', 'Health spa', 'Spa and health club', 'Grocery store', 'Book store', 'Vitamin & supplements store', 'Car wash', 'Banner store', 'Digital printing service' , 'Print shop',  'Market', 'Dollar store',
'Discount store', 'General store', 'Home goods store', 'Office supply store', 'Party store', 'School supply store', 'Variety store', 'Software company', 'Accounting software company', 'Business to business service', 'Computer networking center', 'Financial institution',  'Website designer', 'Performing arts group'
, 'Service establishment', 'Mobile caterer', 'Box lunch supplier', 'Indoor playground',  "Children's party service", 'Park', 'Playground', 'Video arcade', 'Manufacturer', 'Horseback riding service', 'Pony ride service', 'Fresh food market', 'Tourist attraction', 'Weight loss service', 'Eating disorder treatment center'
, 'Wi,Fi spot', 'Butcher shop', 'Hotel',  'Resort hotel',  'Airline ticket agency',  'Courier service', 'Gift shop', 'Money transfer service',   'Shipping service', 'Gym', 'Gymnastics center', 'Parkour spot', 'Rock climbing gym', 'Market operator', 'Grocery delivery service','Appliance repair service'
, 'Store', 'Condiments supplier', 'Recreation center', 'Stadium', 'Plaza', "Children's amusement center", 'Art gallery',   'Wellness program', 'Liquor store',   'Candy store', 'Gas company', 'Wedding venue', 'Hookah store', 'Party planner', 'Produce wholesaler', 'Wholesale grocer'
,  'Drive-in movie theater', 'Coaching center', 'Chocolate shop', 'Corporate office',  'Flower delivery', 'Milk delivery service', 'Adult entertainment club', 'Boutique', 'Clothing store',   'Furniture store', 'Vintage clothing store', 'Shopping mall',   'Movie theater', 'Wholesale market', 'Dry ice supplier'
,   'Chemical wholesaler', 'Industrial gas supplier', 'Stage',   'Amusement center', 'Wedding buffet', 'Food producer']

In [46]:
# Filter the DataFrame to exclude categories_not_to_include
#filtered_df = df6[~df6['category'].apply(lambda x: any(cat in x for cat in categories_not_to_include))]
filtered_df = df6[df6['category'].apply(lambda x: any(cat in x for cat in categories_not_to_include))]

# Display the filtered DataFrame
filtered_df.head()


,business_name,address,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,user_name,time,rating,text
560,Subway,"Subway, 5555 Golden Gate Pkwy Unit 113, Naples, FL 34116","Casual counter-serve chain for build-your-own sandwiches & salads, with health-conscious options.",26.175061,-81.717576,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant'],3.5,98,$,"[[Thursday, 9AM–9PM], [Friday, 9AM–9PM], [Saturday, 9AM–9PM], [Sunday, 9AM–9PM], [Monday, 9AM–9PM], [Tuesday, 9AM–9PM], [Wednesday, 9AM–9PM]]","{'Accessibility': None, 'Activities': None, 'Amenities': None, 'Atmosphere': None, 'Crowd': None, 'Dining options': ['Lunch', 'Catering'], 'From the business': None, 'Getting here': None, 'Health & safety': None, 'Health and safety': None, 'Highlights': None, 'Lodging options': None, 'Offerings'...",Open ⋅ Closes 9PM,Jay Hadding,13/02/2019,4.0,Nana's Diner move next to Subway Subway is actually really good but the picture West from Nana's Diner and they're really good also
561,Subway,"Subway, 5555 Golden Gate Pkwy Unit 113, Naples, FL 34116","Casual counter-serve chain for build-your-own sandwiches & salads, with health-conscious options.",26.175061,-81.717576,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant'],3.5,98,$,"[[Thursday, 9AM–9PM], [Friday, 9AM–9PM], [Saturday, 9AM–9PM], [Sunday, 9AM–9PM], [Monday, 9AM–9PM], [Tuesday, 9AM–9PM], [Wednesday, 9AM–9PM]]","{'Accessibility': None, 'Activities': None, 'Amenities': None, 'Atmosphere': None, 'Crowd': None, 'Dining options': ['Lunch', 'Catering'], 'From the business': None, 'Getting here': None, 'Health & safety': None, 'Health and safety': None, 'Highlights': None, 'Lodging options': None, 'Offerings'...",Open ⋅ Closes 9PM,Janine Dorbad,08/05/2021,5.0,Loved the steak n cheese sub with an array of delicious sauces on your sub
562,Subway,"Subway, 5555 Golden Gate Pkwy Unit 113, Naples, FL 34116","Casual counter-serve chain for build-your-own sandwiches & salads, with health-conscious options.",26.175061,-81.717576,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant'],3.5,98,$,"[[Thursday, 9AM–9PM], [Friday, 9AM–9PM], [Saturday, 9AM–9PM], [Sunday, 9AM–9PM], [Monday, 9AM–9PM], [Tuesday, 9AM–9PM], [Wednesday, 9AM–9PM]]","{'Accessibility': None, 'Activities': None, 'Amenities': None, 'Atmosphere': None, 'Crowd': None, 'Dining options': ['Lunch', 'Catering'], 'From the business': None, 'Getting here': None, 'Health & safety': None, 'Health and safety': None, 'Highlights': None, 'Lodging options': None, 'Offerings'...",Open ⋅ Closes 9PM,KEN ST,14/01/2021,1.0,Very poor customer service. The lady wouldnt get off the phone. I couldnt tell if she was talking to me or the person on the phone. Dont go here. Try slicers on Pineridge.
563,Subway,"Subway, 5555 Golden Gate Pkwy Unit 113, Naples, FL 34116","Casual counter-serve chain for build-your-own sandwiches & salads, with health-conscious options.",26.175061,-81.717576,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant'],3.5,98,$,"[[Thursday, 9AM–9PM], [Friday, 9AM–9PM], [Saturday, 9AM–9PM], [Sunday, 9AM–9PM], [Monday, 9AM–9PM], [Tuesday, 9AM–9PM], [Wednesday, 9AM–9PM]]","{'Accessibility': None, 'Activities': None, 'Amenities': None, 'Atmosphere': None, 'Crowd': None, 'Dining options': ['Lunch', 'Catering'], 'From the business': None, 'Getting here': None, 'Health & safety': None, 'Health and safety': None, 'Highlights': None, 'Lodging options': None, 'Offerings'...",Open ⋅ Closes 9PM,Jordy,05/03/2021,1.0,Little old lady that works here is turning away customers with her attitude. Her coworker is just as bad. No cares given.
564,Subway,"Subway, 5555 Golden Gate Pkwy Unit 113, Naples, FL 34116","Casual counter-serve chain for build-your-own sandwiches & salads, with health-conscious options.",26.175061,-81.717576,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaur

In [47]:
pd.set_option('display.max_colwidth', 300)
# Set the option to display all columns
pd.set_option('display.max_columns', None)

 Pongo a verme los registros a ver si realmente sirven o no esas categorias que a priori parecen erroneas o al menos no de nuestro interes

In [48]:
filtered_df[["business_name", "category"]].head(50)

,business_name,category
560,Subway,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant']
561,Subway,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant']
562,Subway,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant']
563,Subway,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant']
564,Subway,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant']
565,Subway,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant']
566,Subway,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant']
567,Subway,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant']
568,Subway,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant']
569,Subway,['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant']


Aca limpie un poco las categorias que previamente eran hibridas y tras revisar el dataframe anterior filtrado y con gran cantidad de registros, compartian categorias relevantes para nuestra data de restaurantes

In [49]:
categories_not_to_include = ['Event planner', 'Party equipment rental service', 'Gas station', 'Alternative fuel station', 'ATM', 'Electric vehicle charging station', 'Fire protection service', 'HVAC contractor',
'Appliance parts supplier', 'Appliance store', 'Commercial refrigerator supplier', 'Industrial equipment supplier', 'Kitchen supply store', 'Small appliance repair service', 'Used appliance store', 'Wellness center', 'Acupuncture clinic', 'Alternative medicine practitioner', 'Fitness center',
'Hair salon', 'Holistic medicine practitioner', 'Meditation center', 'Pilates studio', 'Yoga studio', 'Health spa', 'Spa and health club', 'Car wash', 'Banner store', 'Digital printing service' , 'Print shop', 'Dollar store',
'Discount store', 'General store', 'Home goods store',  'Party store', 'School supply store', 'Variety store', 'Software company', 'Accounting software company', 'Business to business service', 'Computer networking center', 'Financial institution',  'Website designer', 'Performing arts group'
, 'Service establishment', 'Mobile caterer', 'Box lunch supplier', 'Indoor playground',  "Children's party service", 'Park', 'Playground', 'Video arcade', 'Manufacturer', 'Horseback riding service', 'Pony ride service', 'Tourist attraction', 'Weight loss service', 'Eating disorder treatment center'
, 'Wi,Fi spot', 'Butcher shop', 'Airline ticket agency',  'Courier service', 'Gift shop', 'Money transfer service',   'Shipping service', 'Gym', 'Gymnastics center', 'Parkour spot', 'Rock climbing gym', 'Market operator','Appliance repair service'
,  'Recreation center', 'Stadium', 'Plaza', "Children's amusement center", 'Art gallery',   'Wellness program', 'Liquor store',   'Candy store', 'Gas company', 'Wedding venue', 'Party planner', 'Produce wholesaler',
 'Drive-in movie theater', 'Coaching center', 'Chocolate shop', 'Corporate office',  'Flower delivery', 'Milk delivery service', 'Adult entertainment club', 'Boutique', 'Clothing store',   'Furniture store', 'Vintage clothing store', 'Shopping mall',   'Movie theater', 'Wholesale market', 'Dry ice supplier'
,   'Chemical wholesaler', 'Industrial gas supplier', 'Stage',   'Amusement center', 'Wedding buffet', 'Food producer']

In [50]:
# Filter the DataFrame to exclude categories_not_to_include
#filtered_df = df6[~df6['category'].apply(lambda x: any(cat in x for cat in categories_not_to_include))]
filtered_df = df6[df6['category'].apply(lambda x: any(cat in x for cat in categories_not_to_include))]

# Display the filtered DataFrame
filtered_df
filtered_df[["business_name", "category"]].tail(50)

,business_name,category
1004260,Shell,['Gas station' 'Alternative fuel station' 'ATM' 'Car wash'\n 'Convenience store' 'Electric vehicle charging station' 'Restaurant']
1004261,Shell,['Gas station' 'Alternative fuel station' 'ATM' 'Car wash'\n 'Convenience store' 'Electric vehicle charging station' 'Restaurant']
1004262,Shell,['Gas station' 'Alternative fuel station' 'ATM' 'Car wash'\n 'Convenience store' 'Electric vehicle charging station' 'Restaurant']
1004263,Shell,['Gas station' 'Alternative fuel station' 'ATM' 'Car wash'\n 'Convenience store' 'Electric vehicle charging station' 'Restaurant']
1004264,Shell,['Gas station' 'Alternative fuel station' 'ATM' 'Car wash'\n 'Convenience store' 'Electric vehicle charging station' 'Restaurant']
1004265,Shell,['Gas station' 'Alternative fuel station' 'ATM' 'Car wash'\n 'Convenience store' 'Electric vehicle charging station' 'Restaurant']
1004266,Shell,['Gas station' 'Alternative fuel station' 'ATM' 'Car wash'\n 'Convenience store' 'Electric vehicle charging station' 'Restaurant']
1004267,Shell,['Gas station' 'Alternative fuel station' 'ATM' 'Car wash'\n 'Convenience store' 'Electric vehicle charging station' 'Restaurant']
1004268,Shell,['Gas station' 'Alternative fuel station' 'ATM' 'Car wash'\n 'Convenience store' 'Electric vehicle charging station' 'Restaurant']
1004269,Shell,['Gas station' 'Alternative fuel station' 'ATM' 'Car wash'\n 'Convenience store' 'Electric vehicle charging station' 'Restaurant']


In [51]:
check_cat = df6[["business_name", "category"]]
check_cat[check_cat['category'].apply(lambda x: any(re.search("Book store", category) for category in x))]


,business_name,category


Book store es relevante porque es un cafe con tienda de libros

In [52]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Performing arts group", category) for category in x))]

,business_name,category


Otro caso que es relevante ya que es restaurant con show de arte

In [53]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Service establishment", category) for category in x))]

,business_name,category


Tambien es relevante

In [54]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Mobile caterer", category) for category in x))]

,business_name,category


Relevante

In [55]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Manufacturer", category) for category in x))]

,business_name,category


Relevante

In [56]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Pony ride service", category) for category in x))]#Horseback riding service, Pony ride service

,business_name,category


Relevante

In [57]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Tourist attraction", category) for category in x))]

,business_name,category


Relevabte

In [58]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Weight loss service", category) for category in x))]

,business_name,category


Relevante

In [59]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Eating disorder treatment center", category) for category in x))]

,business_name,category


Relevante

In [60]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Video arcade", category) for category in x))]

,business_name,category


Estan ok son restaruantes con estos servicios extra

In [61]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Butcher shop", category) for category in x))]

,business_name,category


Relevante

In [62]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Rock climbing gym", category) for category in x))]#Gym, Gymnastics center, Parkour spot, Rock climbing gym

,business_name,category


Relevante

In [63]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Market operator", category) for category in x))]

,business_name,category


Relevante

In [64]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Money transfer service", category) for category in x))]

,business_name,category


Relevante

In [65]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Stadium", category) for category in x))]#Sports bar, Recreation center, Stadium

,business_name,category


In [66]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Children's party service", category) for category in x))]#Children's amusement center, Children's party service,  Pizza restaurant, Video arcade

,business_name,category


In [67]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Plaza", category) for category in x))]

,business_name,category


In [68]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Art gallery", category) for category in x))]

,business_name,category


In [69]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Pasta shop", category) for category in x))]

,business_name,category


In [70]:
check_cat[check_cat['category'].apply(lambda x: any(re.search("Wellness program", category) for category in x))]

,business_name,category


Despues de haber checkeado minusiosamnte las categorias que a priori parecian que no habian que incluirlas, viendo los casos pertinentes. Las categorias que deberian quedar serian las siguientes

In [71]:
categories_not_to_include = ['Event planner', 'Party equipment rental service', 'Gas station', 'Alternative fuel station', 'ATM', 'Electric vehicle charging station', 'Fire protection service', 'HVAC contractor',
'Appliance parts supplier', 'Appliance store', 'Commercial refrigerator supplier', 'Industrial equipment supplier', 'Kitchen supply store', 'Small appliance repair service', 'Used appliance store', 'Acupuncture clinic', 'Alternative medicine practitioner', 'Fitness center',
'Hair salon', 'Holistic medicine practitioner', 'Meditation center', 'Pilates studio', 'Yoga studio', 'Health spa', 'Spa and health club', 'Car wash', 'Banner store', 'Digital printing service' , 'Print shop', 'Dollar store',
'Discount store', 'General store', 'Home goods store',  'Party store', 'School supply store', 'Variety store', 'Software company', 'Accounting software company', 'Business to business service', 'Computer networking center', 'Financial institution',  'Website designer', 'Indoor playground',  'Wi,Fi spot', 'Airline ticket agency',  'Courier service', 'Gift shop',   'Shipping service', 'Appliance repair service', 'Liquor store',   'Candy store', 'Gas company', 'Wedding venue', 'Party planner', 'Produce wholesaler',
 'Drive-in movie theater', 'Coaching center', 'Chocolate shop', 'Corporate office',  'Flower delivery', 'Milk delivery service', 'Adult entertainment club', 'Boutique', 'Clothing store',   'Furniture store', 'Vintage clothing store', 'Shopping mall',   'Movie theater', 'Wholesale market', 'Dry ice supplier'
,   'Chemical wholesaler', 'Industrial gas supplier', 'Stage',   'Amusement center', 'Wedding buffet', 'Food producer']


In [72]:
# Reset column width to default (truncate long columns)
pd.reset_option('display.max_colwidth')

# Reset the maximum number of rows displayed (truncation)
pd.reset_option('display.max_rows')

# Reset the maximum number of columns displayed (truncation)
pd.reset_option('display.max_columns')

In [73]:
# Filter the DataFrame to exclude categories_not_to_include
filtered_df = df6[~df6['category'].apply(lambda x: any(cat in x for cat in categories_not_to_include))]


# Display the filtered DataFrame
filtered_df.reset_index()

,index,business_name,address,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,user_name,time,rating,text
0,1,Honduras Nica Cafe,"Honduras Nica Cafe, 280 NW 79th St, Miami, FL ...",None,25.847143,-80.20237,['Honduran restaurant'],3.5,84,None,"[[Thursday, 7AM–9PM], [Friday, 7AM–10PM], [Sat...",{'Accessibility': ['Wheelchair accessible entr...,Closes soon ⋅ 9PM ⋅ Opens 7AM Fri,Cindys 4luvz,17/04/2021,4.0,The nacatamal is a good size and taste ok. The...
1,2,Honduras Nica Cafe,"Honduras Nica Cafe, 280 NW 79th St, Miami, FL ...",None,25.847143,-80.20237,['Honduran restaurant'],3.5,84,None,"[[Thursday, 7AM–9PM], [Friday, 7AM–10PM], [Sat...",{'Accessibility': ['Wheelchair accessible entr...,Closes soon ⋅ 9PM ⋅ Opens 7AM Fri,Ash Jay,07/06/2021,5.0,Love this place thats my favorite place
2,3,Honduras Nica Cafe,"Honduras Nica Cafe, 280 NW 79th St, Miami, FL ...",None,25.847143,-80.20237,['Honduran restaurant'],3.5,84,None,"[[Thursday, 7AM–9PM], [Friday, 7AM–10PM], [Sat...",{'Accessibility': ['Wheelchair accessible entr...,Closes soon ⋅ 9PM ⋅ Opens 7AM Fri,Henmo Mo,10/03/2020,5.0,"Food was fresh, delicious and reminded me of ..."
3,4,Honduras Nica Cafe,"Honduras Nica Cafe, 280 NW 79th St, Miami, FL ...",None,25.847143,-80.20237,['Honduran restaurant'],3.5,84,None,"[[Thursday, 7AM–9PM], [Friday, 7AM–10PM], [Sat...",{'Accessibility': ['Wheelchair accessible entr...,Closes soon ⋅ 9PM ⋅ Opens 7AM Fri,javier tapia,06/09/2020,3.0,Good food great prices
4,5,Honduras Nica Cafe,"Honduras Nica Cafe, 280 NW 79th St, Miami, FL ...",None,25.847143,-80.20237,['Honduran restaurant'],3.5,84,None,"[[Thursday, 7AM–9PM], [Friday, 7AM–10PM], [Sat...",{'Accessibility': ['Wheelchair accessible entr...,Closes soon ⋅ 9PM ⋅ Opens 7AM Fri,Fernando Mejia,02/07/2019,5.0,Best baleadas in town. I always have baleadas ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563222,1005897,Allegria Ristorante,"Allegria Ristorante, 2009 Harrison St, Hollywo...",None,26.010578,-80.14754,['Italian restaurant'],4.6,118,$$,"[[Thursday, 4–10PM], [Friday, 4PM–12AM], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Closes soon ⋅ 10PM ⋅ Opens 4PM Fri,Mary Alonso,01/10/2017,5.0,Exquisite food-great service-perfect ambiance....
563223,1005898,Allegria Ristorante,"Allegria Ristorante, 2009 Harrison St, Hollywo...",None,26.010578,-80.14754,['Italian restaurant'],4.6,118,$$,"[[Thursday, 4–10PM], [Friday, 4PM–12AM], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Closes soon ⋅ 10PM ⋅ Opens 4PM Fri,Pauline Haughton,18/02/2018,5.0,Loved the vegetarian appetizers. Friendly staff
563224,1005899,Allegria Ristorante,"Allegria Ristorante, 2009 Harrison St, Hollywo...",None,26.010578,-80.14754,['Italian restaurant'],4.6,118,$$,"[[Thursday, 4–10PM], [Friday, 4PM–12AM], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Closes soon ⋅ 10PM ⋅ Opens 4PM Fri,Ashley Davis,28/02/2018,5.0,Love this place.
563225,1005900,Allegria Ristorante,"Allegria Ristorante, 2009 Harrison St, Hollywo...",None,26.010578,-80.14754,['Italian restaurant'],4.6,118,$$,"[[Thursday, 4–10PM], [Friday, 4PM–12AM], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Closes soon ⋅ 10PM ⋅ Opens 4PM Fri,Michel Comtois,24/02/2019,4.0,(Translated by Google) Music too loud.\n\n(Ori...


In [74]:
filtered_df.columns

Index(['business_name', 'address', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC',
       'state', 'user_name', 'time', 'rating', 'text'],
      dtype='object')

In [75]:
filtered_df.avg_rating.value_counts()

4.4    80251
4.5    71485
4.3    60991
4.2    59779
4.6    50241
4.1    41828
4.0    33775
3.9    24353
4.7    23801
3.8    20700
3.7    19721
3.5    15404
3.6    15143
3.3    10708
4.8     9474
3.4     8301
3.2     4076
4.9     3452
3.1     3366
2.9     1917
3.0     1861
2.8     1043
5.0      883
2.6      210
2.7      130
2.4       69
2.2       58
2.1       58
2.5       44
1.8       37
2.3       26
1.7       18
1.5       12
1.9       12
Name: avg_rating, dtype: int64

In [76]:
filtered_df.num_of_reviews.value_counts()

8116    3462
38      3310
58      3153
48      2614
848     2597
        ... 
69        13
22        13
23        13
41        13
32         8
Name: num_of_reviews, Length: 878, dtype: int64

In [77]:
filtered_df.price.value_counts()

₩₩      167759
₩       140868
$       105069
$$       90311
₩₩₩      10797
$$$       3621
₩₩₩₩      3518
$$$$       453
Name: price, dtype: int64

In [78]:
filtered_df.MISC

1          {'Accessibility': ['Wheelchair accessible entr...
2          {'Accessibility': ['Wheelchair accessible entr...
3          {'Accessibility': ['Wheelchair accessible entr...
4          {'Accessibility': ['Wheelchair accessible entr...
5          {'Accessibility': ['Wheelchair accessible entr...
                                 ...                        
1005897    {'Accessibility': ['Wheelchair accessible entr...
1005898    {'Accessibility': ['Wheelchair accessible entr...
1005899    {'Accessibility': ['Wheelchair accessible entr...
1005900    {'Accessibility': ['Wheelchair accessible entr...
1005901    {'Accessibility': ['Wheelchair accessible entr...
Name: MISC, Length: 563227, dtype: object

HAY QUE DESANIDAR ESTQA COLUMNA


In [79]:
filtered_df.to_parquet('FINAL_FLORIDA.parquet')


In [ ]:
df6.shape

In [80]:
filtered_df.shape

(563227, 16)

In [81]:
filtered_df_unique = filtered_df.drop_duplicates(subset='address', keep='first')
filtered_df_unique.shape
#cantridad de restaurants

(3743, 16)